In [ ]:
!pip install geopandas
!pip install folium
!pip install lightgbm
!pip install utm
!pip install geojson 
!pip install haversine
!pip install pickle5

In [ ]:
# 시각화 도구 설치
!pip install plotnine #R 시각화 
!pip install folium #지도 시각화 

# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [1]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [2]:
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import numpy as np
import folium 
import geopandas as gpd
import math
from tqdm.notebook import tqdm
import utm
from shapely.geometry import Point, Polygon
from haversine import haversine
from plotnine import *
import pickle5 as pickle

In [24]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(school_loc, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('지역아동센터')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(school_loc['학교명'],school_loc['위도'],school_loc['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['Latitude'],target['Longitude']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

## 0. 학교데이터로 Target 만들기

In [5]:
# 학교데이터 불러오기 
with open('/content/drive/My Drive/Project/Seoul/초등학교/초중고위치.pkl', 'rb') as f:  
  df_school = pickle.load(f)

#### 학교데이터 전처리 

In [6]:
mask = df_school['학교급구분'].str.contains('초등') 
df_school = df_school[mask].reset_index().drop(['index'], axis=1)

In [7]:
df_school['위도'] = df_school['위도'].astype(float)
df_school['경도'] = df_school['경도'].astype(float)

# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_school['경도'], df_school['위도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
df_school = gpd.GeoDataFrame(df_school, crs=crs, geometry=geometry)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
df_school = df_school.drop('동',axis=1)
df_school

,학교명,학교급구분,소재지지번주소,소재지도로명주소,위도,경도,구,법정동코드,법정동,행정동코드,행정동,geometry
0,서울휘경초등학교,초등학교,서울특별시 동대문구 휘경동 302-2,서울특별시 동대문구 망우로6길 48,37.585736,127.058213,동대문구,1123010900,휘경동.1,1123073000,휘경제2동,POINT (127.05821 37.58574)
1,서울동답초등학교,초등학교,서울특별시 동대문구 답십리동 2-24,서울특별시 동대문구 답십리로60길 12,37.571503,127.064034,동대문구,1123010500,답십리동,1123061000,답십리제2동,POINT (127.06403 37.57150)
2,서울답십리초등학교,초등학교,서울특별시 동대문구 답십리동 274,서울특별시 동대문구 전농로3길 23,37.568804,127.055429,동대문구,1123010500,답십리동,1123060000,답십리제1동,POINT (127.05543 37.56880)
3,서울신답초등학교,초등학교,서울특별시 동대문구 답십리동 463-13,서울특별시 동대문구 천호대로 177,37.572705,127.042962,동대문구,1123010500,답십리동,1123060000,답십리제1동,POINT (127.04296 37.57271)
4,서울용두초등학교,초등학교,서울특별시 동대문구 용두동 237-1,서울특별시 동대문구 천호대로13길 43,37.576415,127.028387,동대문구,1123010200,용두동,1123053600,용신동,POINT (127.02839 37.57641)
...,...,...,...,...,...,...,...,...,...,...,...,...
598,서울위례별초등학교,초등학교,서울특별시 송파구 장지동 886,서울특별시 송파구 위례광장로 243,37.479086,127.145042,송파구,1171010900,장지동,1171064700,위례동,POINT (127.14504 37.47909)
599,서울하늘숲초등학교,초등학교,서울특별시 구로구 천왕동 산13-1,서울특별시 구로구 오리로 1100,37.483405,126.842135,구로구,1153011100,천왕동,1153078000,오류제2동,POINT (126.84213 37.48340)
600,서울언남초등학교,초등학교,서울특별시 서초구 내곡동 121,서울특별시 서초구 헌릉로8길 42,37.453898,127.061885,서초구,1165010900,내곡동,1165066000,내곡동,POINT (127.06189 37.45390)
601,서울항동초등학교,초등학교,서울특별시 구로구 항동 160,서울특별시 구로구 연동로 178,37.476972,126.824078,구로구,1153011200,항동,1153080000,항동,POINT (126.82408 37.47697)


In [9]:
# df_school.to_pickle('/content/drive/My Drive/Project/Seoul/초등학교/초등학교.pickle')

### 학교데이터로 Target 만들기

In [10]:
taregt_school = df_school[['행정동코드','구', '행정동','학교명','소재지도로명주소', '위도', '경도', 'geometry']]
taregt_school

,행정동코드,구,행정동,학교명,소재지도로명주소,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,37.585736,127.058213,POINT (127.05821 37.58574)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,37.571503,127.064034,POINT (127.06403 37.57150)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,37.568804,127.055429,POINT (127.05543 37.56880)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,37.572705,127.042962,POINT (127.04296 37.57271)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,37.576415,127.028387,POINT (127.02839 37.57641)
...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,37.479086,127.145042,POINT (127.14504 37.47909)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,37.483405,126.842135,POINT (126.84213 37.48340)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,37.453898,127.061885,POINT (127.06189 37.45390)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,37.476972,126.824078,POINT (126.82408 37.47697)


In [11]:
merge_base = taregt_school[['학교명']]
merge_base

,학교명
0,서울휘경초등학교
1,서울동답초등학교
2,서울답십리초등학교
3,서울신답초등학교
4,서울용두초등학교
...,...
598,서울위례별초등학교
599,서울하늘숲초등학교
600,서울언남초등학교
601,서울항동초등학교


## 1. 우리마을키움센터

In [137]:
df_dolbom = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/키움센터_위치_행정동.geojson')
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),22,4,서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),30,6,서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),80,16,서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),40,8,서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),40,8,서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),20,4,서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),25,5,서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),20,4,서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),20,4,서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


#### 전처리

In [12]:
df_dolbom = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson')
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),22,4,서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),30,6,서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),80,16,서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),40,8,서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),40,8,서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),20,4,서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),25,5,서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),20,4,서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),20,4,서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [ ]:
# 우리동네 키움포털 
df_crawl = pd.read_csv('/content/drive/My Drive/Project/Seoul/돌봄시설 api/우리동네키움포털.csv').drop(['Unnamed: 0','주소'], axis=1)
df_crawl['시설ID'] = df_crawl['센터'].map(lambda x: x.split(' ')[0])
df_crawl

,센터,상시돌봄,일시돌봄,시설ID
0,강남1호점 우리동네키움센터강남 다함께키움센터태화,45,3,강남1호점
1,강남2호점 우리동네키움센터강남 다함께키움센터역삼,20,4,강남2호점
2,강남3호점 우리동네키움센터강남 다함께키움센터청담,20,4,강남3호점
3,강남4호점 우리동네키움센터미미위 세곡 키움센터,25,5,강남4호점
4,강동1호점 우리동네키움센터상상나루來키움센터,20,4,강동1호점
...,...,...,...,...
86,중구2호점 우리동네키움센터모든아이 중림센터,40,8,중구2호점
87,중구3호점 우리동네키움센터모든아이 손기정센터,40,8,중구3호점
88,중구4호점 우리동네키움센터모든아이 약수센터,80,16,중구4호점
89,중구5호점 우리동네키움센터모든아이 신당5동센터,30,6,중구5호점


In [ ]:
# # 돌봄센터 ID 만들기 

# df_dolbom['시설ID'] = df_dolbom['시설명'].map(lambda x: x.split(' ')[0])
# df_dolbom['시설ID'].unique()
# df_dolbom = df_dolbom[['자치구', '행정동코드', '법정동', '행정동', '시설ID','시설명', '주소', 'Latitude', 'Longitude',
#        'geometry']]
# df_dolbom.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson', driver='GeoJSON')
# df_dolbom.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.csv', index=False)

In [ ]:
df_merge = df_dolbom.merge(df_crawl, how ='left', on ='시설ID').drop('센터', axis=1)
df_dolbom = df_merge[['자치구', '행정동코드', '법정동', '행정동', '시설ID', '시설명', '상시돌봄', '일시돌봄','주소', 'Latitude',
       'Longitude', 'geometry']]
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),22,4,서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),30,6,서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),80,16,서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),40,8,서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),40,8,서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),20,4,서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),25,5,서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),20,4,서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),20,4,서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [136]:
# df_dolbom.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/키움센터_위치_행정동.geojson', driver='GeoJSON')
# df_dolbom.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/키움센터_위치_행정동.csv', index=False)

### 거리

In [14]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(school_loc, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('parking_nm')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(school_loc['학교명'],school_loc['위도'],school_loc['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['Latitude'],target['Longitude']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

In [80]:
lscp_dolbom=pd.DataFrame(harv_dis(taregt_school,df_dolbom,881), columns=['학교명','키움센터','키움센터_거리'])
lscp_dolbom

,학교명,키움센터,키움센터_거리
0,서울동답초등학교,동대문2호점,639.866446
1,서울신답초등학교,성동8호점,844.669250
2,서울신답초등학교,성동3호점,506.946984
3,서울용두초등학교,성동8호점,817.863932
4,서울용두초등학교,동대문1호점,172.838994
...,...,...,...
352,서울하늘숲초등학교,구로5호점,521.321341
353,서울숭신초등학교,중구5호점,612.779818
354,서울숭신초등학교,성동8호점,598.008884
355,서울숭신초등학교,성동2호점,816.383363


In [81]:
lscp_dolbom = lscp_dolbom.merge(df_dolbom, left_on='키움센터', right_on='시설ID')[['학교명', '키움센터', '키움센터_거리', '상시돌봄', '일시돌봄']]
lscp_dolbom

,학교명,키움센터,키움센터_거리,상시돌봄,일시돌봄
0,서울동답초등학교,동대문2호점,639.866446,20,4
1,서울장평초등학교,동대문2호점,483.550086,20,4
2,서울배봉초등학교,동대문2호점,510.392213,20,4
3,서울전동초등학교,동대문2호점,801.489103,20,4
4,은석초등학교,동대문2호점,679.988401,20,4
...,...,...,...,...,...
352,서울오현초등학교,성북1호점,799.894528,29,5
353,서울우솔초등학교,서초1호점,248.268192,20,4
354,서울자곡초등학교,강남4호점,639.998460,25,5
355,서울율현초등학교,강남4호점,171.993913,25,5


In [73]:
# 키움센터 하나에 여러 학교 
lscp_dolbom.groupby(['키움센터','학교명']).agg('mean')

키움센터_거리  상시돌봄  일시돌봄
키움센터  학교명                             
강남1호점 서울대모초등학교  801.467851    45     3
      서울왕북초등학교  673.274058    45     3
강남2호점 서울역삼초등학교  732.487180    20     4
강남3호점 서울봉은초등학교  601.073326    20     4
      서울언북초등학교  830.713113    20     4
...                    ...   ...   ...
중구5호점 서울무학초등학교  713.176809    30     6
      서울숭신초등학교  612.779818    30     6
      서울신당초등학교  228.519713    30     6
      서울흥인초등학교  796.039353    30     6
중랑1호점 서울새솔초등학교  413.691542    22     4

[357 rows x 3 columns]

In [74]:
# 학교 하나에 여러 키움센터 
lscp_dolbom.groupby(['학교명', '키움센터']).agg('mean')

키움센터_거리  상시돌봄  일시돌봄
학교명    키움센터                          
광운초등학교 노원1호점   396.628858    20     4
       노원4호점   404.378192    20     4
       성북1호점   787.214751    29     5
대광초등학교 동대문1호점  694.076713    30     6
동산초등학교 중구1호점   608.415921    30     6
...                   ...   ...   ...
청원초등학교 노원3호점   439.522567    28     5
       노원9호점   737.410728    20     4
충암초등학교 은평3호점   640.953500    20     4
한양초등학교 성동9호점   554.853600    15     3
화랑초등학교 노원6호점   787.040932    20     4

[357 rows x 3 columns]

In [94]:
#초등학교별로 groupby 
df_loc = pd.DataFrame(lscp_dolbom, columns = ['학교명','키움센터','키움센터_거리']) 
lscp_dolbom_groupby = dict(list(df_loc.groupby('학교명')))

# 전체 초등학교 리스트를 데이터 프레임으로 만들기 
df_dolbom_loc = pd.DataFrame(lscp_dolbom_groupby.items())
df_dolbom_loc['학교명'] = np.array(np.nan)

# 각 초등학교의 가까운 키움센터 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_dolbom_groupby.keys()):
  stations.append(lscp_dolbom_groupby[k]['키움센터'].values)

#초등학교와 각 초등학교별 키움센터 데이터 프레임 합치기 
df_dolbom_loc['키움센터'] = stations
df_dolbom_loc = df_dolbom_loc[[0, '키움센터']]
df_dolbom_loc.columns = ['학교명', '키움센터']
df_dolbom_loc

,학교명,키움센터
0,광운초등학교,"[노원1호점, 노원4호점, 성북1호점]"
1,대광초등학교,[동대문1호점]
2,동산초등학교,"[중구1호점, 중구4호점]"
3,명지초등학교,[은평3호점]
4,상명초등학교,"[노원5호점, 노원18호점, 노원17호점, 노원19호점]"
...,...,...
226,은석초등학교,[동대문2호점]
227,청원초등학교,"[노원2호점, 노원3호점, 노원22호점, 노원21호점, 노원16호점, 노원9호점]"
228,충암초등학교,[은평3호점]
229,한양초등학교,[성동9호점]


In [96]:
df_dolbom_loc['키움센터_상시돌봄정원'] = list(lscp_dolbom.groupby(['학교명']).agg('sum')['상시돌봄'])
df_dolbom_loc['키움센터_일시돌봄정원'] = list(lscp_dolbom.groupby(['학교명']).agg('sum')['일시돌봄'])
df_dolbom_loc

,학교명,키움센터,키움센터_상시돌봄정원,키움센터_일시돌봄정원
0,광운초등학교,"[노원1호점, 노원4호점, 성북1호점]",69,13
1,대광초등학교,[동대문1호점],30,6
2,동산초등학교,"[중구1호점, 중구4호점]",110,22
3,명지초등학교,[은평3호점],20,4
4,상명초등학교,"[노원5호점, 노원18호점, 노원17호점, 노원19호점]",109,21
...,...,...,...,...
226,은석초등학교,[동대문2호점],20,4
227,청원초등학교,"[노원2호점, 노원3호점, 노원22호점, 노원21호점, 노원16호점, 노원9호점]",135,26
228,충암초등학교,[은평3호점],20,4
229,한양초등학교,[성동9호점],15,3


In [97]:
df_dolbom_loc = merge_base.merge(df_dolbom_loc,on='학교명', how='outer')
df_dolbom_loc

,학교명,키움센터,키움센터_상시돌봄정원,키움센터_일시돌봄정원
0,서울휘경초등학교,NaN,NaN,NaN
1,서울동답초등학교,[동대문2호점],20.0,4.0
2,서울답십리초등학교,NaN,NaN,NaN
3,서울신답초등학교,"[성동8호점, 성동3호점]",47.0,9.0
4,서울용두초등학교,"[성동8호점, 동대문1호점]",50.0,10.0
...,...,...,...,...
598,서울위례별초등학교,NaN,NaN,NaN
599,서울하늘숲초등학교,[구로5호점],34.0,7.0
600,서울언남초등학교,NaN,NaN,NaN
601,서울항동초등학교,NaN,NaN,NaN


## 초등돌봄센터

In [ ]:
# element = pd.read_excel("/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.xlsx")
# element

In [98]:
gdf_element = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.geojson')
gdf_element

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476971,126.824078,POINT (37.477 126.824)


#### 초등돌봄 전처리 

In [25]:
element = pd.read_excel("/content/drive/My Drive/Project/Seoul/초등학교/초등돌봄교실이용현황_서울.xlsx")
element

,시군구,학교명,교실수,이용인원,소재지
0,종로구,서울교동초등학교,3,50,서울특별시 종로구 삼일대로 446 (경운동)
1,종로구,서울독립문초등학교,4,80,서울특별시 종로구 통일로12길 23 (무악동)
2,종로구,서울매동초등학교,3,65,서울특별시 종로구 사직로9길 19 . 서울매동초등학교 (필운동)
3,종로구,서울명신초등학교,3,60,서울특별시 종로구 낙산길 250 (창신동)
4,종로구,서울세검정초등학교,5,111,서울특별시 종로구 세검정로9길 1 (신영동.서울세검정초등학교)
...,...,...,...,...,...
567,강동구,서울신암초등학교,4,100,서울특별시 강동구 올림픽로93길 33-16 . 서울신암초등학교 (암사동)
568,강동구,서울천동초등학교,4,89,서울특별시 강동구 성안로 200 . 서울천동초등학교 (길동)
569,강동구,서울천일초등학교,2,45,서울특별시 강동구 구천면로 266 . 서울천일초등학교 (천호동)
570,강동구,서울천호초등학교,3,74,서울특별시 강동구 구천면로 310-11 (천호동.서울천호초등학교)


In [26]:
## 초등돌봄시설 전처리 
element = pd.read_excel("/content/drive/My Drive/Project/Seoul/초등학교/초등돌봄교실이용현황_서울.xlsx")
# element = element.drop(['연번', '시도', '비고'], axis = 1)

# 서울염강초등학교 폐지 후 서울가양초등학교 및 서울염경초등학교로 통･폐합
mask = element['학교명'].str.contains('염강') 
element = element[~mask].reset_index().drop(['index'], axis=1)

# element.to_excel('/content/drive/My Drive/Project/Seoul/초등학교/초등돌봄교실이용현황_서울.xlsx', index = False)

In [27]:
len(element['학교명'].unique())

572

In [28]:
# 서울염강초등학교 폐지 후 서울가양초등학교 및 서울염경초등학교로 통･폐합
for e in element['학교명']:
  if e not in taregt_school['학교명'].values:
    print(e)

In [29]:
taregt_school[taregt_school['학교명'].str.contains('염강')]

,행정동코드,구,행정동,학교명,소재지도로명주소,위도,경도,geometry


#### 위치코드와 합치기 

In [30]:
df_element = taregt_school.merge(element, on='학교명',how='left').drop(['소재지','시군구'], axis=1).fillna(0)
df_element = df_element[['행정동코드', '구', '행정동', '학교명', '소재지도로명주소','교실수', '이용인원', '위도', '경도', 'geometry']]
df_element

,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (127.05821 37.58574)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (127.06403 37.57150)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (127.05543 37.56880)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (127.04296 37.57271)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (127.02839 37.57641)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (127.14504 37.47909)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (126.84213 37.48340)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (127.06189 37.45390)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (126.82408 37.47697)


In [31]:
df_element[df_element['이용인원'].isnull()]

,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry


In [32]:
# 결측치 0으로 넣었음 
df_element[df_element['이용인원']==0]

,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry
42,1126069000,중랑구,신내2동,금성초등학교,서울특별시 중랑구 신내로21길 55,0.0,0.0,37.615314,127.089859,POINT (127.08986 37.61531)
70,1138055200,은평구,갈현제2동,선일초등학교,서울특별시 은평구 갈현로29길 27,0.0,0.0,37.619716,126.912800,POINT (126.91280 37.61972)
71,1138056000,은평구,구산동,예일초등학교,서울특별시 은평구 연서로 117,0.0,0.0,37.609642,126.916096,POINT (126.91610 37.60964)
72,1138052000,은평구,불광제1동,은혜초등학교,서울특별시 은평구 통일로78가길 20,0.0,0.0,37.617733,126.927400,POINT (126.92740 37.61773)
87,1141056500,서대문구,충현동,경기초등학교,서울특별시 서대문구 경기대로9길 10,0.0,0.0,37.565110,126.962861,POINT (126.96286 37.56511)
88,1141058500,서대문구,신촌동,이화여자대학교사범대학부속초등학교,서울특별시 서대문구 성산로 512-39,0.0,0.0,37.561149,126.942750,POINT (126.94275 37.56115)
89,1141056500,서대문구,충현동,추계초등학교,서울특별시 서대문구 북아현로11가길 7,0.0,0.0,37.563150,126.953209,POINT (126.95321 37.56315)
90,1141068500,서대문구,홍은제2동,명지초등학교,서울특별시 서대문구 명지2길 32-20,0.0,0.0,37.583630,126.928230,POINT (126.92823 37.58363)
153,1154568000,금천구,시흥제2동,동광초등학교,서울특별시 금천구 탑골로10길 45,0.0,0.0,37.451672,126.916887,POINT (126.91689 37.45167)
242,1111064000,종로구,이화동,서울대학교사범대학부설초등학교,서울특별시 종로구 대학로 64,0.0,0.0,37.577600,127.004297,POINT (127.00430 37.57760)


In [ ]:
df_element.to_excel('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.xlsx', index = False)

In [33]:
df_element = df_element.drop(['geometry'], axis=1)
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_element['위도'], df_element['경도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_element = gpd.GeoDataFrame(df_element, crs=crs, geometry=geometry)
gdf_element

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (37.477 126.824)


In [34]:
gdf_element.columns = ['행정동코드', '구', '행정동', '학교명', '소재지도로명주소', '돌봄교실_개수', '돌봄교실_이용인원', '위도', '경도',
       'geometry']
gdf_element

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (37.477 126.824)


In [35]:
gdf_element.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.geojson', driver='GeoJSON')

## 지역아동센터

In [49]:
df_local =  gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/지역아동센터_위치포함.geojson')
df_local

,행정동코드,시군구,행정동,시설명,입소정원(명),시설 소재지,Latitude,Longitude,geometry
0,1111067000,종로구,창신제1동,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319,POINT (127.01332 37.57312)
1,1111068000,종로구,창신제2동,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336,POINT (127.01034 37.57730)
2,1111064000,종로구,이화동,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640,POINT (127.00864 37.57776)
3,1111068000,종로구,창신제2동,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204,POINT (127.00920 37.57509)
4,1111070000,종로구,숭인제1동,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016529,POINT (127.01653 37.57785)
...,...,...,...,...,...,...,...,...,...
451,1174065000,강동구,성내제2동,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897,POINT (127.12790 37.53537)
452,1174057000,강동구,암사제1동,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245,POINT (127.12925 37.55055)
453,1174066000,강동구,성내제3동,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566,POINT (127.13357 37.52823)
454,0,강동구,0,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241,POINT (127.18224 37.55207)


In [25]:
# df_local = pd.read_excel('/content/drive/My Drive/Project/Seoul/초등학교/지역아동센터_191231기준.xlsx')
# df_local

,시도,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude
0,서울,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319
1,서울,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336
2,서울,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640
3,서울,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204
4,서울,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528
...,...,...,...,...,...,...,...
451,서울,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897
452,서울,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245
453,서울,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566
454,서울,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241


#### 전처리

In [26]:
df_local = df_local.drop(['시도'],axis = 1)
df_local

,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude
0,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319
1,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336
2,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640
3,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204
4,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528
...,...,...,...,...,...,...
451,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897
452,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245
453,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566
454,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241


In [27]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [28]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_local['Longitude'], df_local['Latitude'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_local = gpd.GeoDataFrame(df_local, crs=crs, geometry=geometry)
gdf_local

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude,geometry
0,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319,POINT (127.01332 37.57312)
1,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336,POINT (127.01034 37.57730)
2,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640,POINT (127.00864 37.57776)
3,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204,POINT (127.00920 37.57509)
4,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528,POINT (127.01653 37.57785)
...,...,...,...,...,...,...,...
451,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897,POINT (127.12790 37.53537)
452,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245,POINT (127.12925 37.55055)
453,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566,POINT (127.13357 37.52823)
454,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241,POINT (127.18224 37.55207)


In [29]:
# 행정동 포함여부 확인 
gdf_local['행정동코드'] = np.zeros(len(gdf_local))
gdf_local['행정동'] = np.zeros(len(gdf_local))

for i in gdf_local.index:
    for j in dong_geo.index:
        if gdf_local.geometry[i].within(dong_geo.geometry[j]):
            gdf_local['행정동'][i] = dong_geo.읍면동명[j]
            gdf_local['행정동코드'][i] = dong_geo.행정동코드[j]
            
gdf_local['행정동코드'] = gdf_local['행정동코드'].astype(int)
gdf_local

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude,geometry,행정동코드,행정동
0,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319,POINT (127.01332 37.57312),1111067000,창신제1동
1,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336,POINT (127.01034 37.57730),1111068000,창신제2동
2,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640,POINT (127.00864 37.57776),1111064000,이화동
3,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204,POINT (127.00920 37.57509),1111068000,창신제2동
4,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528,POINT (127.01653 37.57785),1111070000,숭인제1동
...,...,...,...,...,...,...,...,...,...
451,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897,POINT (127.12790 37.53537),1174065000,성내제2동
452,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245,POINT (127.12925 37.55055),1174057000,암사제1동
453,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566,POINT (127.13357 37.52823),1174066000,성내제3동
454,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241,POINT (127.18224 37.55207),0,0


In [47]:
gdf_local = gdf_local[['행정동코드', '시군구','행정동', '시설명', '입소정원(명)', '시설 소재지', 'Latitude', 'Longitude', 'geometry']]
gdf_local

,행정동코드,시군구,행정동,시설명,입소정원(명),시설 소재지,Latitude,Longitude,geometry
0,1111067000,종로구,창신제1동,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319,POINT (127.01332 37.57312)
1,1111068000,종로구,창신제2동,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336,POINT (127.01034 37.57730)
2,1111064000,종로구,이화동,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640,POINT (127.00864 37.57776)
3,1111068000,종로구,창신제2동,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204,POINT (127.00920 37.57509)
4,1111070000,종로구,숭인제1동,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528,POINT (127.01653 37.57785)
...,...,...,...,...,...,...,...,...,...
451,1174065000,강동구,성내제2동,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897,POINT (127.12790 37.53537)
452,1174057000,강동구,암사제1동,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245,POINT (127.12925 37.55055)
453,1174066000,강동구,성내제3동,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566,POINT (127.13357 37.52823)
454,0,강동구,0,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241,POINT (127.18224 37.55207)


In [48]:
gdf_local.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/지역아동센터_위치포함.geojson', driver='GeoJSON')

### 거리

In [99]:
lscp_local=pd.DataFrame(harv_dis(taregt_school,df_local,881), columns=['학교명','지역아동센터','돌봄센터_거리'])
lscp_local

,학교명,지역아동센터,돌봄센터_거리
0,서울휘경초등학교,염광지역아동센터빛나는집,850.813117
1,서울휘경초등학교,해피아트지역아동센터,266.110861
2,서울휘경초등학교,원광지역아동센터,722.309281
3,서울휘경초등학교,나누리지역아동센터,824.852274
4,서울동답초등학교,염광지역아동센터빛나는집,819.583702
...,...,...,...
1682,서울하늘숲초등학교,구로구공립지역아동센터,475.358542
1683,서울숭신초등학교,엘림지역아동센터,863.171727
1684,서울숭신초등학교,조이스터디,669.820637
1685,서울숭신초등학교,성동지역아동복지센터,807.600973


In [107]:
lscp_local = lscp_local.merge(df_local, left_on='지역아동센터', right_on='시설명')[['학교명','지역아동센터','돌봄센터_거리', '입소정원(명)']]
lscp_local

,학교명,지역아동센터,돌봄센터_거리,입소정원(명)
0,서울휘경초등학교,염광지역아동센터빛나는집,850.813117,49
1,서울동답초등학교,염광지역아동센터빛나는집,819.583702,49
2,서울배봉초등학교,염광지역아동센터빛나는집,457.355956,49
3,서울전동초등학교,염광지역아동센터빛나는집,112.672353,49
4,서울전곡초등학교,염광지역아동센터빛나는집,571.981754,49
...,...,...,...,...
2018,서울인수초등학교,성심중앙지역아동센터,385.293267,30
2019,서울우이초등학교,성심중앙지역아동센터,640.409838,30
2020,서울우이초등학교,산지역아동센터,438.137816,19
2021,서울세명초등학교,LH행복꿈터열린마음지역아동센터,180.432312,19


In [108]:
# 돌봄센터 하나에 여러 학교 
lscp_local.groupby(['지역아동센터','학교명']).agg('mean')

돌봄센터_거리  입소정원(명)
지역아동센터           학교명                          
1318해피존새움터지역아동센터 서울구남초등학교  837.289475     19.0
                 서울성자초등학교  339.970343     19.0
                 서울신자초등학교  833.378772     19.0
                 서울양남초등학교  643.584364     19.0
                 서울자양초등학교  418.287768     19.0
...                               ...      ...
희망의집지역아동센터       서울수유초등학교  361.894776     40.0
                 서울유현초등학교  737.416801     40.0
희망지역아동센터         서울신강초등학교  847.963519     19.0
                 서울신정초등학교  531.971856     19.0
                 서울양동초등학교  877.255376     19.0

[1687 rows x 2 columns]

In [109]:
# 학교 하나에 여러 돌봄센터 
lscp_local.groupby(['학교명', '지역아동센터']).agg('mean')

돌봄센터_거리  입소정원(명)
학교명             지역아동센터                                
경기초등학교          구세군지역아동복지센터        442.000864     50.0
                나무를심는학교            756.187664     29.0
                등대지역아동센터           732.545209     29.0
                신일지역아동센터           350.628703     19.0
                청소년1318해피존나무를심는학교  581.317492     19.0
...                                       ...      ...
한신초등학교          즐거운지역아동센터          296.882035     29.5
한양초등학교          성동좋은이웃지역아동센터       634.295683     35.0
홍익대학교사범대학부속초등학교 망원무지개지역아동센터        821.639618     49.0
                옹달샘지역아동센터          822.461879     31.0
                행복무지개공부방지역아동센터     272.517655     19.0

[1687 rows x 2 columns]

In [113]:
#초등학교별로 groupby 
df_loc = pd.DataFrame(lscp_local, columns = ['학교명','지역아동센터','지역아동센터_거리']) 
lscp_local_groupby = dict(list(df_loc.groupby('학교명')))

# 전체 초등학교 리스트를 데이터 프레임으로 만들기 
df_local_loc = pd.DataFrame(lscp_local_groupby.items())
df_local_loc['학교명'] = np.array(np.nan)

# 각 초등학교의 가까운 지역아동센터 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_local_groupby.keys()):
  stations.append(lscp_local_groupby[k]['지역아동센터'].values)

#초등학교와 각 초등학교별 지역아동센터 데이터 프레임 합치기 
df_local_loc['지역아동센터'] = stations
df_local_loc = df_local_loc[[0, '지역아동센터']]
df_local_loc.columns = ['학교명', '지역아동센터']
df_local_loc

,학교명,지역아동센터
0,경기초등학교,"[등대지역아동센터, 신일지역아동센터, 구세군지역아동복지센터, 청소년1318해피존나무..."
1,경복초등학교,"[행복한지역아동센터, 행복한지역아동센터, 행복한지역아동센터, 행복한지역아동센터]"
2,경희초등학교,"[해피아트지역아동센터, 행복한지역아동센터, 행복한지역아동센터, 행복한지역아동센터, ..."
3,광운초등학교,"[구립월계문화지역아동센터, 장위지역아동센터밀알학교, 푸른하늘지역아동센터, 다솔지역아..."
4,금성초등학교,[마루아라지역아동센터]
...,...,...
498,추계초등학교,"[신일지역아동센터, 구세군지역아동복지센터, 청소년1318해피존나무를심는학교, 나무를..."
499,충암초등학교,"[꿈이있는푸른학교지역아동센터, 비전스쿨지역아동센터, 새로핌지역아동센터, 광암지역아동센터]"
500,한신초등학교,"[즐거운지역아동센터, 즐거운지역아동센터, 무지개지역아동센터, 무지개지역아동센터, 방..."
501,한양초등학교,[성동좋은이웃지역아동센터]


In [114]:
df_local_loc['지역아동센터_정원'] = list(lscp_local.groupby(['학교명']).agg('sum')['입소정원(명)'])
df_local_loc

,학교명,지역아동센터,지역아동센터_정원
0,경기초등학교,"[등대지역아동센터, 신일지역아동센터, 구세군지역아동복지센터, 청소년1318해피존나무...",146
1,경복초등학교,"[행복한지역아동센터, 행복한지역아동센터, 행복한지역아동센터, 행복한지역아동센터]",129
2,경희초등학교,"[해피아트지역아동센터, 행복한지역아동센터, 행복한지역아동센터, 행복한지역아동센터, ...",235
3,광운초등학교,"[구립월계문화지역아동센터, 장위지역아동센터밀알학교, 푸른하늘지역아동센터, 다솔지역아...",145
4,금성초등학교,[마루아라지역아동센터],21
...,...,...,...
498,추계초등학교,"[신일지역아동센터, 구세군지역아동복지센터, 청소년1318해피존나무를심는학교, 나무를...",117
499,충암초등학교,"[꿈이있는푸른학교지역아동센터, 비전스쿨지역아동센터, 새로핌지역아동센터, 광암지역아동센터]",123
500,한신초등학교,"[즐거운지역아동센터, 즐거운지역아동센터, 무지개지역아동센터, 무지개지역아동센터, 방...",292
501,한양초등학교,[성동좋은이웃지역아동센터],35


In [115]:
df_local_loc = merge_base.merge(df_local_loc,on='학교명', how='outer')
df_local_loc

,학교명,지역아동센터,지역아동센터_정원
0,서울휘경초등학교,"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터...",181.0
1,서울동답초등학교,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]",126.0
2,서울답십리초등학교,"[장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터, 성동좋은이웃지역아동센터]",112.0
3,서울신답초등학교,[조이스터디],32.0
4,서울용두초등학교,[아름다운안암지역아동센터],22.0
...,...,...,...
598,서울위례별초등학교,NaN,NaN
599,서울하늘숲초등학교,"[구로구공립지역아동센터, 예지지역아동센터]",54.0
600,서울언남초등학교,NaN,NaN
601,서울항동초등학교,NaN,NaN


## 방과후아카데미

In [116]:
df_after =  gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/방과후아카데미_위치포함.geojson')
df_after

,행정동코드,시군구,행정동,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude,geometry
0,1114065000,중구,신당제5동,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022778,POINT (127.02278 37.56505)
1,1114063500,중구,약수동,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
2,1120056000,성동구,행당제1동,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712,POINT (127.03671 37.56306)
3,1123070500,동대문구,청량리동,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988,POINT (127.04999 37.58727)
4,1126065500,중랑구,망우본동,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848,POINT (127.10885 37.60588)
5,1126054000,중랑구,면목제4동,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839,POINT (127.08584 37.57317)
6,1129078000,성북구,장위제3동,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734,POINT (127.05573 37.61757)
7,1130564500,강북구,우이동,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380,POINT (127.00638 37.64594)
8,1132051400,도봉구,창제4동,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615,POINT (127.05061 37.65827)
9,1138059000,은평구,응암제2동,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524,POINT (126.92452 37.59415)


In [41]:
df_after = pd.read_excel('/content/drive/My Drive/Project/Seoul/초등학교/청소년방과후아카데미_초등부_200130기준.xlsx')
df_after

,연번,시도,시군구,시설명,운영유형,초등학생 수용인원(정원),소재지,Latitude,Longitude
0,1,서울,중구,유락종합사회복지관,기본지원형,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777
1,2,서울,중구,서울중구청소년수련관,기본지원형,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577
2,3,서울,성동구,시립성동청소년센터,기본지원형,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712
3,4,서울,동대문구,시립동대문청소년센터,기본지원형,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988
4,5,서울,중랑구,서울시립망우청소년센터,기본지원형,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848
5,6,서울,중랑구,시립중랑청소년센터,기본지원형,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839
6,7,서울,성북구,시립성북청소년센터(초등),기본지원형,40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734
7,8,서울,강북구,시립강북청소년센터,기본지원형,40,서울특별시 강북구 4.19로 74,37.645936,127.006380
8,9,서울,도봉구,"시립창동청소년센터(초등,중등)",기본지원형,20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615
9,10,서울,은평구,시립은평청소년센터,기본지원형,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524


#### 전처리

In [42]:
df_after = df_after.drop(['시도','연번', '운영유형'],axis = 1)
df_after

,시군구,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude
0,중구,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777
1,중구,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577
2,성동구,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712
3,동대문구,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988
4,중랑구,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848
5,중랑구,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839
6,성북구,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734
7,강북구,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380
8,도봉구,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615
9,은평구,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524


In [43]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [44]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_after['Longitude'], df_after['Latitude'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_after = gpd.GeoDataFrame(df_after, crs=crs, geometry=geometry)
gdf_after

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,시군구,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude,geometry
0,중구,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777,POINT (127.02278 37.56505)
1,중구,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577,POINT (127.01258 37.55178)
2,성동구,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712,POINT (127.03671 37.56306)
3,동대문구,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988,POINT (127.04999 37.58727)
4,중랑구,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848,POINT (127.10885 37.60588)
5,중랑구,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839,POINT (127.08584 37.57317)
6,성북구,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734,POINT (127.05573 37.61757)
7,강북구,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380,POINT (127.00638 37.64594)
8,도봉구,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615,POINT (127.05061 37.65827)
9,은평구,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524,POINT (126.92452 37.59415)


In [45]:
# 행정동 포함여부 확인 
gdf_after['행정동코드'] = np.zeros(len(gdf_after))
gdf_after['행정동'] = np.zeros(len(gdf_after))

for i in gdf_after.index:
    for j in dong_geo.index:
        if gdf_after.geometry[i].within(dong_geo.geometry[j]):
            gdf_after['행정동'][i] = dong_geo.읍면동명[j]
            gdf_after['행정동코드'][i] = dong_geo.행정동코드[j]
            
gdf_after['행정동코드'] = gdf_after['행정동코드'].astype(int)
gdf_after

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,시군구,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude,geometry,행정동코드,행정동
0,중구,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777,POINT (127.02278 37.56505),1114065000,신당제5동
1,중구,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577,POINT (127.01258 37.55178),1114063500,약수동
2,성동구,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712,POINT (127.03671 37.56306),1120056000,행당제1동
3,동대문구,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988,POINT (127.04999 37.58727),1123070500,청량리동
4,중랑구,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848,POINT (127.10885 37.60588),1126065500,망우본동
5,중랑구,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839,POINT (127.08584 37.57317),1126054000,면목제4동
6,성북구,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734,POINT (127.05573 37.61757),1129078000,장위제3동
7,강북구,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380,POINT (127.00638 37.64594),1130564500,우이동
8,도봉구,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615,POINT (127.05061 37.65827),1132051400,창제4동
9,은평구,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524,POINT (126.92452 37.59415),1138059000,응암제2동


In [51]:
gdf_after = gdf_after[['행정동코드','시군구', '행정동', '시설명', '초등학생 수용인원(정원)', '소재지', 'Latitude', 'Longitude',
       'geometry']]
gdf_after

,행정동코드,시군구,행정동,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude,geometry
0,1114065000,중구,신당제5동,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777,POINT (127.02278 37.56505)
1,1114063500,중구,약수동,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577,POINT (127.01258 37.55178)
2,1120056000,성동구,행당제1동,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712,POINT (127.03671 37.56306)
3,1123070500,동대문구,청량리동,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988,POINT (127.04999 37.58727)
4,1126065500,중랑구,망우본동,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848,POINT (127.10885 37.60588)
5,1126054000,중랑구,면목제4동,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839,POINT (127.08584 37.57317)
6,1129078000,성북구,장위제3동,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734,POINT (127.05573 37.61757)
7,1130564500,강북구,우이동,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380,POINT (127.00638 37.64594)
8,1132051400,도봉구,창제4동,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615,POINT (127.05061 37.65827)
9,1138059000,은평구,응암제2동,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524,POINT (126.92452 37.59415)


In [52]:
gdf_after.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/방과후아카데미_위치포함.geojson', driver='GeoJSON')

### 거리

In [117]:
lscp_after=pd.DataFrame(harv_dis(taregt_school,df_after,881), columns=['학교명','방과후아카데미','방과후아카데미_거리'])
lscp_after

,학교명,방과후아카데미,방과후아카데미_거리
0,서울휘경초등학교,시립동대문청소년센터,744.512486
1,서울홍릉초등학교,시립동대문청소년센터,430.109136
2,서울청량초등학교,시립동대문청소년센터,765.678122
3,경희초등학교,시립동대문청소년센터,859.344208
4,서울삼육초등학교,시립동대문청소년센터,298.955468
...,...,...,...
62,서울장곡초등학교,시립성북청소년센터(초등),672.468199
63,광운초등학교,시립성북청소년센터(초등),174.876372
64,서울인수초등학교,시립강북청소년센터,402.239741
65,서울강솔초등학교,시립강동청소년센터,811.662989


In [120]:
lscp_after = lscp_after.merge(df_after, left_on='방과후아카데미', right_on='시설명')[['학교명','방과후아카데미','방과후아카데미_거리', '초등학생 수용인원(정원)']]
lscp_after

,학교명,방과후아카데미,방과후아카데미_거리,초등학생 수용인원(정원)
0,서울휘경초등학교,시립동대문청소년센터,744.512486,40
1,서울홍릉초등학교,시립동대문청소년센터,430.109136,40
2,서울청량초등학교,시립동대문청소년센터,765.678122,40
3,경희초등학교,시립동대문청소년센터,859.344208,40
4,서울삼육초등학교,시립동대문청소년센터,298.955468,40
...,...,...,...,...
62,서울동명초등학교,시립성동청소년센터,581.057857,40
63,서울마장초등학교,시립성동청소년센터,754.251117,40
64,서울무학초등학교,시립성동청소년센터,626.195501,40
65,서울행현초등학교,시립성동청소년센터,878.935237,40


In [121]:
# 방과후아카데미 하나에 여러 학교 
lscp_after.groupby(['방과후아카데미','학교명']).agg('mean')

방과후아카데미_거리  초등학생 수용인원(정원)
방과후아카데미       학교명                                
마포구립망원청소년문화센터 서울동교초등학교  372.337711             10
              서울망원초등학교  470.597189             10
서울시립망우청소년센터   서울동원초등학교  410.406832             20
              서울신내초등학교  775.259464             20
서울중구청소년수련관    동산초등학교    613.242766             20
...                            ...            ...
유락종합사회복지관     서울금북초등학교  842.966503             20
              서울무학초등학교  634.191531             20
              서울숭신초등학교  545.572611             20
              서울신당초등학교  177.417227             20
              서울흥인초등학교  861.232840             20

[67 rows x 2 columns]

In [122]:
# 학교 하나에 여러 방과후아카데미 
lscp_after.groupby(['학교명', '방과후아카데미']).agg('mean')

,,방과후아카데미_거리,초등학생 수용인원(정원)
학교명,방과후아카데미,,
경희초등학교,시립동대문청소년센터,859.344208,40
광운초등학교,시립성북청소년센터(초등),174.876372,40
동산초등학교,서울중구청소년수련관,613.242766,20
서울가인초등학교,"시립창동청소년센터(초등,중등)",687.491941,20
서울강솔초등학교,시립강동청소년센터,811.662989,40
...,...,...,...
서울행당초등학교,시립성동청소년센터,690.454404,40
서울행현초등학교,시립성동청소년센터,878.935237,40
서울홍릉초등학교,시립동대문청소년센터,430.109136,40


In [123]:
#초등학교별로 groupby 
df_loc = pd.DataFrame(lscp_after, columns = ['학교명','방과후아카데미','방과후아카데미_거리']) 
lscp_after_groupby = dict(list(df_loc.groupby('학교명')))

# 전체 초등학교 리스트를 데이터 프레임으로 만들기 
df_after_loc = pd.DataFrame(lscp_after_groupby.items())
df_after_loc['학교명'] = np.array(np.nan)

# 각 초등학교의 가까운 방과후아카데미 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_after_groupby.keys()):
  stations.append(lscp_after_groupby[k]['방과후아카데미'].values)

#초등학교와 각 초등학교별 방과후아카데미 데이터 프레임 합치기 
df_after_loc['방과후아카데미'] = stations
df_after_loc = df_after_loc[[0, '방과후아카데미']]
df_after_loc.columns = ['학교명', '방과후아카데미']
df_after_loc

,학교명,방과후아카데미
0,경희초등학교,[시립동대문청소년센터]
1,광운초등학교,[시립성북청소년센터(초등)]
2,동산초등학교,[서울중구청소년수련관]
3,서울가인초등학교,"[시립창동청소년센터(초등,중등)]"
4,서울강솔초등학교,[시립강동청소년센터]
...,...,...
59,서울행당초등학교,[시립성동청소년센터]
60,서울행현초등학교,[시립성동청소년센터]
61,서울홍릉초등학교,[시립동대문청소년센터]
62,서울휘경초등학교,[시립동대문청소년센터]


In [124]:
df_after_loc['방과후아카데미_정원'] = list(lscp_after.groupby(['학교명']).agg('sum')['초등학생 수용인원(정원)'])
df_after_loc

,학교명,방과후아카데미,방과후아카데미_정원
0,경희초등학교,[시립동대문청소년센터],40
1,광운초등학교,[시립성북청소년센터(초등)],40
2,동산초등학교,[서울중구청소년수련관],20
3,서울가인초등학교,"[시립창동청소년센터(초등,중등)]",20
4,서울강솔초등학교,[시립강동청소년센터],40
...,...,...,...
59,서울행당초등학교,[시립성동청소년센터],40
60,서울행현초등학교,[시립성동청소년센터],40
61,서울홍릉초등학교,[시립동대문청소년센터],40
62,서울휘경초등학교,[시립동대문청소년센터],40


In [125]:
df_after_loc = merge_base.merge(df_after_loc,on='학교명', how='outer')
df_after_loc

,학교명,방과후아카데미,방과후아카데미_정원
0,서울휘경초등학교,[시립동대문청소년센터],40.0
1,서울동답초등학교,NaN,NaN
2,서울답십리초등학교,NaN,NaN
3,서울신답초등학교,NaN,NaN
4,서울용두초등학교,NaN,NaN
...,...,...,...
598,서울위례별초등학교,NaN,NaN
599,서울하늘숲초등학교,NaN,NaN
600,서울언남초등학교,NaN,NaN
601,서울항동초등학교,NaN,NaN


## Merge

In [126]:
df_after_loc

,학교명,방과후아카데미,방과후아카데미_정원
0,서울휘경초등학교,[시립동대문청소년센터],40.0
1,서울동답초등학교,NaN,NaN
2,서울답십리초등학교,NaN,NaN
3,서울신답초등학교,NaN,NaN
4,서울용두초등학교,NaN,NaN
...,...,...,...
598,서울위례별초등학교,NaN,NaN
599,서울하늘숲초등학교,NaN,NaN
600,서울언남초등학교,NaN,NaN
601,서울항동초등학교,NaN,NaN


In [127]:
df_local_loc

,학교명,지역아동센터,지역아동센터_정원
0,서울휘경초등학교,"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터...",181.0
1,서울동답초등학교,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]",126.0
2,서울답십리초등학교,"[장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터, 성동좋은이웃지역아동센터]",112.0
3,서울신답초등학교,[조이스터디],32.0
4,서울용두초등학교,[아름다운안암지역아동센터],22.0
...,...,...,...
598,서울위례별초등학교,NaN,NaN
599,서울하늘숲초등학교,"[구로구공립지역아동센터, 예지지역아동센터]",54.0
600,서울언남초등학교,NaN,NaN
601,서울항동초등학교,NaN,NaN


In [128]:
gdf_element

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476971,126.824078,POINT (37.477 126.824)


In [129]:
df_dolbom_loc

,학교명,키움센터,키움센터_상시돌봄정원,키움센터_일시돌봄정원
0,서울휘경초등학교,NaN,NaN,NaN
1,서울동답초등학교,[동대문2호점],20.0,4.0
2,서울답십리초등학교,NaN,NaN,NaN
3,서울신답초등학교,"[성동8호점, 성동3호점]",47.0,9.0
4,서울용두초등학교,"[성동8호점, 동대문1호점]",50.0,10.0
...,...,...,...,...
598,서울위례별초등학교,NaN,NaN,NaN
599,서울하늘숲초등학교,[구로5호점],34.0,7.0
600,서울언남초등학교,NaN,NaN,NaN
601,서울항동초등학교,NaN,NaN,NaN


In [130]:
df_1 = gdf_element.merge(df_dolbom_loc, on='학교명')
df_2 = df_1.merge(df_after_loc, on='학교명')
df_3 = df_2.merge(df_local_loc, on='학교명')

df_3

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry,키움센터,키움센터_상시돌봄정원,키움센터_일시돌봄정원,방과후아카데미,방과후아카데미_정원,지역아동센터,지역아동센터_정원
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058),NaN,NaN,NaN,[시립동대문청소년센터],40.0,"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터...",181.0
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064),[동대문2호점],20.0,4.0,NaN,NaN,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]",126.0
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055),NaN,NaN,NaN,NaN,NaN,"[장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터, 성동좋은이웃지역아동센터]",112.0
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043),"[성동8호점, 성동3호점]",47.0,9.0,NaN,NaN,[조이스터디],32.0
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028),"[성동8호점, 동대문1호점]",50.0,10.0,NaN,NaN,[아름다운안암지역아동센터],22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145),NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842),[구로5호점],34.0,7.0,NaN,NaN,"[구로구공립지역아동센터, 예지지역아동센터]",54.0
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062),NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476971,126.824078,POINT (37.477 126.824),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
df_3.columns

Index(['행정동코드', '구', '행정동', '학교명', '소재지도로명주소', '돌봄교실_개수', '돌봄교실_이용인원', '위도',
       '경도', 'geometry', '키움센터', '키움센터_상시돌봄정원', '키움센터_일시돌봄정원', '방과후아카데미',
       '방과후아카데미_정원', '지역아동센터', '지역아동센터_정원'],
      dtype='object')

In [133]:
df_final = df_3[['행정동코드', '구', '행정동', '학교명', '소재지도로명주소', '키움센터', '키움센터_상시돌봄정원', '키움센터_일시돌봄정원',
                 '돌봄교실_개수', '돌봄교실_이용인원', '방과후아카데미',
       '방과후아카데미_정원',  '지역아동센터', '지역아동센터_정원', '위도',
       '경도', 'geometry']]
df_final

,행정동코드,구,행정동,학교명,소재지도로명주소,키움센터,키움센터_상시돌봄정원,키움센터_일시돌봄정원,돌봄교실_개수,돌봄교실_이용인원,방과후아카데미,방과후아카데미_정원,지역아동센터,지역아동센터_정원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,NaN,NaN,NaN,3.0,72.0,[시립동대문청소년센터],40.0,"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터...",181.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,[동대문2호점],20.0,4.0,4.0,93.0,NaN,NaN,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]",126.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,NaN,NaN,NaN,4.0,89.0,NaN,NaN,"[장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터, 성동좋은이웃지역아동센터]",112.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,"[성동8호점, 성동3호점]",47.0,9.0,4.0,92.0,NaN,NaN,[조이스터디],32.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,"[성동8호점, 동대문1호점]",50.0,10.0,4.0,73.0,NaN,NaN,[아름다운안암지역아동센터],22.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,NaN,NaN,NaN,7.0,121.0,NaN,NaN,NaN,NaN,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,[구로5호점],34.0,7.0,3.0,75.0,NaN,NaN,"[구로구공립지역아동센터, 예지지역아동센터]",54.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,NaN,NaN,NaN,7.0,174.0,NaN,NaN,NaN,NaN,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,NaN,NaN,NaN,4.0,45.0,NaN,NaN,NaN,NaN,37.476971,126.824078,POINT (37.477 126.824)


In [139]:
# geojson은 numpy가 들어갈수없다고 합니다...
# pickle로 일단 만들었습니다. 
df_final.to_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/학교별_주변공적돌봄시설.pickle')

In [ ]:
df_school = pd.read_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/학교별_주변공적돌봄시설.pickle')
df_school

In [142]:
df_supply_num = df_school.drop(['키움센터', '방과후아카데미','지역아동센터'],axis=1)
df_supply_num['공적돌봄'] = df_school['키움센터_상시돌봄정원'] +df_school['키움센터_일시돌봄정원']+df_school['방과후아카데미_정원']+df_school['돌봄교실_이용인원']+df_school['지역아동센터_정원']
df_supply_num

,행정동코드,구,행정동,학교명,소재지도로명주소,키움센터_상시돌봄정원,키움센터_일시돌봄정원,돌봄교실_개수,돌봄교실_이용인원,방과후아카데미_정원,지역아동센터_정원,위도,경도,geometry,공적돌봄
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,NaN,NaN,3.0,72.0,40.0,181.0,37.585736,127.058213,POINT (37.586 127.058),NaN
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,20.0,4.0,4.0,93.0,NaN,126.0,37.571503,127.064034,POINT (37.572 127.064),NaN
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,NaN,NaN,4.0,89.0,NaN,112.0,37.568804,127.055429,POINT (37.569 127.055),NaN
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,47.0,9.0,4.0,92.0,NaN,32.0,37.572705,127.042962,POINT (37.573 127.043),NaN
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,50.0,10.0,4.0,73.0,NaN,22.0,37.576415,127.028387,POINT (37.576 127.028),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,NaN,NaN,7.0,121.0,NaN,NaN,37.479086,127.145042,POINT (37.479 127.145),NaN
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,34.0,7.0,3.0,75.0,NaN,54.0,37.483405,126.842135,POINT (37.483 126.842),NaN
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,NaN,NaN,7.0,174.0,NaN,NaN,37.453898,127.061885,POINT (37.454 127.062),NaN
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,NaN,NaN,4.0,45.0,NaN,NaN,37.476971,126.824078,POINT (37.477 126.824),NaN


## 학교기준 EDA

In [159]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [172]:
df_school = pd.read_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/학교별_주변공적돌봄시설.pickle')

In [174]:
df_school = df_school.fillna(0)
df_school['공적돌봄'] = df_school['키움센터_상시돌봄정원'] +df_school['키움센터_일시돌봄정원']+df_school['방과후아카데미_정원']+df_school['돌봄교실_이용인원']+df_school['지역아동센터_정원']
df_school

,행정동코드,구,행정동,학교명,소재지도로명주소,키움센터,키움센터_상시돌봄정원,키움센터_일시돌봄정원,돌봄교실_개수,돌봄교실_이용인원,방과후아카데미,방과후아카데미_정원,지역아동센터,지역아동센터_정원,위도,경도,geometry,공적돌봄
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,0,0.0,0.0,3.0,72.0,[시립동대문청소년센터],40.0,"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터...",181.0,37.585736,127.058213,POINT (37.586 127.058),293.0
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,[동대문2호점],20.0,4.0,4.0,93.0,0,0.0,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]",126.0,37.571503,127.064034,POINT (37.572 127.064),243.0
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,0,0.0,0.0,4.0,89.0,0,0.0,"[장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터, 성동좋은이웃지역아동센터]",112.0,37.568804,127.055429,POINT (37.569 127.055),201.0
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,"[성동8호점, 성동3호점]",47.0,9.0,4.0,92.0,0,0.0,[조이스터디],32.0,37.572705,127.042962,POINT (37.573 127.043),180.0
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,"[성동8호점, 동대문1호점]",50.0,10.0,4.0,73.0,0,0.0,[아름다운안암지역아동센터],22.0,37.576415,127.028387,POINT (37.576 127.028),155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,0,0.0,0.0,7.0,121.0,0,0.0,0,0.0,37.479086,127.145042,POINT (37.479 127.145),121.0
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,[구로5호점],34.0,7.0,3.0,75.0,0,0.0,"[구로구공립지역아동센터, 예지지역아동센터]",54.0,37.483405,126.842135,POINT (37.483 126.842),170.0
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,0,0.0,0.0,7.0,174.0,0,0.0,0,0.0,37.453898,127.061885,POINT (37.454 127.062),174.0
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,0,0.0,0.0,4.0,45.0,0,0.0,0,0.0,37.476971,126.824078,POINT (37.477 126.824),45.0


In [158]:
df_supply = df_school.groupby('행정동코드', as_index=False).agg('sum').drop(['위도', '경도'],axis=1)
df_supply

,행정동코드,키움센터_상시돌봄정원,키움센터_일시돌봄정원,돌봄교실_개수,돌봄교실_이용인원,방과후아카데미_정원,지역아동센터_정원,공적돌봄
0,1111051500,20.0,4.0,5.0,149.0,0.0,0.0,173.0
1,1111053000,20.0,4.0,3.0,65.0,0.0,54.0,143.0
2,1111055000,0.0,0.0,5.0,111.0,0.0,0.0,111.0
3,1111057000,0.0,0.0,4.0,80.0,0.0,123.0,203.0
4,1111060000,0.0,0.0,2.0,34.0,0.0,0.0,34.0
...,...,...,...,...,...,...,...,...
357,1174060000,40.0,8.0,5.0,119.0,0.0,459.0,626.0
358,1174061000,20.0,4.0,4.0,100.0,0.0,178.0,302.0
359,1174064000,0.0,0.0,10.0,252.0,0.0,237.0,489.0
360,1174068500,0.0,0.0,13.0,255.0,0.0,593.0,848.0


In [171]:
df_supply = df_school.groupby('행정동코드', as_index=False).agg('sum').drop(['위도', '경도'],axis=1)
df_supply['행정동코드'] = df_supply['행정동코드'].astype(int)
dong_geo['행정동코드'] = dong_geo['행정동코드'].astype(int)

df_supply = dong_geo.merge(df_supply, on='행정동코드')
df_supply

,행정동코드,시군구명,읍면동명,위도,경도,geometry,키움센터_상시돌봄정원,키움센터_일시돌봄정원,돌봄교실_개수,돌봄교실_이용인원,방과후아카데미_정원,지역아동센터_정원,공적돌봄
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",20.0,4.0,3.0,65.0,0.0,54.0,143.0
1,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ...",0.0,0.0,5.0,111.0,0.0,0.0,111.0
2,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ...",0.0,0.0,4.0,80.0,0.0,123.0,203.0
3,1111060000,종로구,가회동,37.582676,126.986618,"MULTIPOLYGON (((126.98914 37.59131, 126.98831 ...",0.0,0.0,2.0,34.0,0.0,0.0,34.0
4,1111061500,종로구,종로1.2.3.4가동,37.575077,126.989733,"MULTIPOLYGON (((126.99650 37.58102, 126.99641 ...",0.0,0.0,3.0,50.0,0.0,0.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ...",0.0,0.0,7.0,153.0,0.0,203.0,356.0
357,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ...",0.0,0.0,4.0,97.0,0.0,114.0,211.0
358,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ...",20.0,4.0,4.0,100.0,0.0,178.0,302.0
359,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ...",0.0,0.0,13.0,255.0,0.0,593.0,848.0


In [188]:
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),22,4,서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),30,6,서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),80,16,서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),40,8,서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),40,8,서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),20,4,서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),25,5,서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),20,4,서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),20,4,서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [191]:
gdf_element

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476971,126.824078,POINT (37.477 126.824)


In [192]:
import folium 
from folium import plugins

center = (37.5642135, 127.0016985)
zoom = 14

m = folium.Map(location=center, zoom_start=11)
choropleth = folium.Choropleth(
    geo_data = dong_geo,
    data = df_supply,
    columns = ('읍면동명', '공적돌봄'),
    key_on = 'feature.properties.읍면동명',
    fill_color = 'BuPu',
    legend_name = 'None'
).add_to(m)

for index, row in df_dolbom.iterrows():
    icon_number = plugins.BeautifyIcon( border_color='#48D9CA', text_color='#48D9CA', number=1, inner_icon_style='margin-top:0;')
    folium.Marker([row['Latitude'], row['Longitude']], tooltip='dolbom', icon=icon_number).add_to(m)  

for index, row in gdf_element.iterrows():
    icon_number = plugins.BeautifyIcon( border_color='#32BAD9', text_color='#32BAD9', number=2, inner_icon_style='margin-top:0;')
    folium.Marker([row['위도'], row['경도']], tooltip='chodeung', icon=icon_number).add_to(m)  

for index, row in df_local.iterrows():
    icon_number = plugins.BeautifyIcon( border_color='#30A8F2', text_color='#30A8F2', number=3, inner_icon_style='margin-top:0;')
    folium.Marker([row['Latitude'], row['Longitude']], tooltip='local center', icon=icon_number).add_to(m)  

for index, row in df_after.iterrows():
    icon_number = plugins.BeautifyIcon( border_color='#8480F2', text_color='#8480F2', number=4, inner_icon_style='margin-top:0;')
    folium.Marker([row['Latitude'], row['Longitude']], tooltip='after school academy', icon=icon_number).add_to(m)  


for index, row in df_school.iterrows():
  folium.CircleMarker([row['위도'], row['경도']], radius = 0.00000002*row['공적돌봄'], color = 'red').add_to(m)  
  # folium.Marker([row['위도'], row['경도']]).add_to(m)

m